In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\Users\\anura\\Gen AI project\\Medical-Chat-Bot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\anura\\Gen AI project\\Medical-Chat-Bot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf(data='Data/')

In [8]:
extracted_data

[Document(metadata={'producer': 'Corel PDF Engine Version 16.0.0.707', 'creator': 'CorelDRAW X6', 'creationdate': '2017-06-16T12:56:12+05:30', 'author': 'Prakash', 'moddate': '2018-05-04T17:18:10+01:00', 'title': 'Untitled-1', 'source': 'Data\\medical_book_2.pdf', 'total_pages': 52, 'page': 0, 'page_label': '1'}, page_content="NOSISA SYSTEM OF DIA  \nIN OUTLINE\nProf. Ashok Chandra\nMD, MRCP (UK), FRCP, FICP, FIACM \nFormer Professor & Head of Medicine \nKing George's Medical University \nProfessor & Head of Medicine \nEmeritus Professor of Medicine \nEra's Lucknow Medical College \nLucknow, India\nHistory\nExamination\nGeneral Exam\nSystem Involved\nOther Systems\nInvestigations\nSite\nEtiology\nPathophysiology\nDysfunction\nCLINICAL\nMEDICINE\nG NOSISDIA"),
 Document(metadata={'producer': 'Corel PDF Engine Version 16.0.0.707', 'creator': 'CorelDRAW X6', 'creationdate': '2017-06-16T12:56:12+05:30', 'author': 'Prakash', 'moddate': '2018-05-04T17:18:10+01:00', 'title': 'Untitled-1', 'so

In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [10]:
text_chunks = text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 185


In [11]:
text_chunks

[Document(metadata={'producer': 'Corel PDF Engine Version 16.0.0.707', 'creator': 'CorelDRAW X6', 'creationdate': '2017-06-16T12:56:12+05:30', 'author': 'Prakash', 'moddate': '2018-05-04T17:18:10+01:00', 'title': 'Untitled-1', 'source': 'Data\\medical_book_2.pdf', 'total_pages': 52, 'page': 0, 'page_label': '1'}, page_content="NOSISA SYSTEM OF DIA  \nIN OUTLINE\nProf. Ashok Chandra\nMD, MRCP (UK), FRCP, FICP, FIACM \nFormer Professor & Head of Medicine \nKing George's Medical University \nProfessor & Head of Medicine \nEmeritus Professor of Medicine \nEra's Lucknow Medical College \nLucknow, India\nHistory\nExamination\nGeneral Exam\nSystem Involved\nOther Systems\nInvestigations\nSite\nEtiology\nPathophysiology\nDysfunction\nCLINICAL\nMEDICINE\nG NOSISDIA"),
 Document(metadata={'producer': 'Corel PDF Engine Version 16.0.0.707', 'creator': 'CorelDRAW X6', 'creationdate': '2017-06-16T12:56:12+05:30', 'author': 'Prakash', 'moddate': '2018-05-04T17:18:10+01:00', 'title': 'Untitled-1', 'so

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


c:\Users\anura\Gen AI project\Medical-Chat-Bot\med_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\anura\Gen AI project\Medical-Chat-Bot\med_env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\anura\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to ru

In [13]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

In [31]:
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [21]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

In [22]:
index_name = 'medchatbot'

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medchatbot",
    "metric": "cosine",
    "host": "medchatbot-jdu3u3h.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [32]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
from langchain_pinecone import PineconeVectorStore
doc_search = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [25]:
# Load existing index
from langchain_pinecone import PineconeVectorStore
doc_search = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [26]:
doc_search

In [27]:
retriever = doc_search.as_retriever(search_type="similarity", search_kwargs = {"k":3})

In [28]:
retrieved_docs = retriever.invoke("What is hemopytsis")

In [29]:
retrieved_docs

[Document(id='6993677a-b90f-4d2c-88f3-29a29227ee77', metadata={'author': 'Prakash', 'creationdate': '2017-06-16T12:56:12+05:30', 'creator': 'CorelDRAW X6', 'moddate': '2018-05-04T17:18:10+01:00', 'page': 20.0, 'page_label': '21', 'producer': 'Corel PDF Engine Version 16.0.0.707', 'source': 'Data\\medical_book_2.pdf', 'title': 'Untitled-1', 'total_pages': 52.0}, page_content='etiological diagnosis as the causes of hemoptysis at each sub\xadsite of the respiratory system are few and many \ndiseases have their distinctive characteristics.\n        ~ Upper Respiratory Tract often gives a Streaky  hemoptysis\n        ~ Alveolar origin of hemoptysis is often Frothy  and is a hallmark of pulmonary edema\n18'),
 Document(id='5edf6377-efdc-4124-8fe5-f797e1f5c003', metadata={'author': 'Prakash', 'creationdate': '2017-06-16T12:56:12+05:30', 'creator': 'CorelDRAW X6', 'moddate': '2018-05-04T17:18:10+01:00', 'page': 21.0, 'page_label': '22', 'producer': 'Corel PDF Engine Version 16.0.0.707', 'sourc

In [33]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt_template = (
    "You are an assistant for question answering tasks"
    "Use the following pieces of retrieved context to answer"
    "In any question if you don't know the answer, say I don't know"
    "Give answers only in three lines to make it more concise"
    "Try to give the answer in simple way so that anyone can understand. Also under 3 lines"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt_template),
        ("human", "{input}")
    ]
)

In [35]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [41]:
response = rag_chain.invoke({"input": "what is acromegaly and gigantism"})
print(response["answer"])


Acromegaly is a disorder caused by excessive growth hormone production, leading to enlargement of the hands, feet, and facial features. Gigantism is a similar disorder, but it occurs before the growth plates have closed, resulting in excessive height in addition to other symptoms. Both are endocrinal manifestations that can be caused by underlying conditions such as Cushings disease or tumors.
